# 🤖 Agent系统调试笔记本

本笔记本用于调试和测试Agent系统：
- Agent路由测试
- Agent加载和配置
- 消息处理流程
- Agent组合测试

In [ ]:
# 环境设置
import sys
import os
from pathlib import Path

# 添加项目根目录到路径
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

# 设置必要的环境变量（测试用）
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

print(f"项目根目录: {project_root}")
print("✅ 环境设置完成")

## 1. 加载Agent模块

In [ ]:
from agents.emotional_agent import EmotionalAgent
from agents.tech_agent import TechAgent
from agents.tool_agent import ToolAgent

print("📦 已加载的Agent:")
print("   • EmotionalAgent - 情感支持Agent")
print("   • TechAgent - 技术问题Agent")
print("   • ToolAgent - 工具调用Agent")

## 2. Agent实例化和配置

In [ ]:
# 查看Agent文件内容
agents_dir = project_root / "agents"

print("📁 Agents目录内容:")
for f in agents_dir.iterdir():
    if f.is_file() and f.suffix == ".py":
        print(f"   • {f.name}")

In [ ]:
# 查看EmotionalAgent源码
emotional_agent_file = agents_dir / "emotional_agent.py"
if emotional_agent_file.exists():
    content = emotional_agent_file.read_text(encoding="utf-8")
    print("📄 EmotionalAgent 源码 (前50行):")
    print("=" * 60)
    lines = content.split("\n")[:50]
    for i, line in enumerate(lines, 1):
        print(f"{i:3}. {line}")

## 3. Agent路由测试

In [ ]:
# 定义关键词路由规则
AGENT_ROUTING_RULES = {
    "EmotionalAgent": {
        "keywords": ["难过", "开心", "焦虑", "压力", "心情", "情绪", "孤独", "伤心", "担心"],
        "priority": 100,
    },
    "TechAgent": {
        "keywords": ["代码", "编程", "Python", "JavaScript", "技术", "bug", "报错", "API"],
        "priority": 80,
    },
    "ToolAgent": {
        "keywords": ["计算", "翻译", "搜索", "查询", "工具"],
        "priority": 60,
    },
}

def route_message(message: str) -> str:
    """根据消息内容路由到合适的Agent"""
    scores = {}
    
    for agent_name, rules in AGENT_ROUTING_RULES.items():
        score = 0
        for keyword in rules["keywords"]:
            if keyword.lower() in message.lower():
                score += rules["priority"]
        scores[agent_name] = score
    
    if max(scores.values()) == 0:
        return "EmotionalAgent"  # 默认
    
    return max(scores, key=scores.get)

# 测试路由
test_messages = [
    "今天心情不太好，感觉很焦虑",
    "Python代码报错了，帮我看看",
    "帮我计算一下这个数学题",
    "今天天气真好",
    "我想学习编程，但是感觉很难过",
]

print("🧭 Agent路由测试:")
print("=" * 60)
for msg in test_messages:
    agent = route_message(msg)
    print(f"\n💬 \"{msg}\"")
    print(f"   ➡️  {agent}")

## 4. Agent配置从Bot加载

In [ ]:
from src.bot.config_loader import BotConfigLoader

# 加载所有Bot配置
loader = BotConfigLoader(str(project_root / "bots"))
configs = loader.load_all_configs()

print("🤖 Bot的Agent配置:")
print("=" * 60)

for bot_id, config in configs.items():
    print(f"\n📌 {bot_id} ({config.name})")
    print(f"   Agents: {[a.name for a in config.agents.enabled]}")
    print(f"   Fallback: {config.agents.fallback}")

## 5. Agent响应模拟

In [ ]:
# 模拟Agent响应（不实际调用API）
class MockAgent:
    def __init__(self, name: str, style: str):
        self.name = name
        self.style = style
    
    def generate_response_template(self, message: str) -> str:
        """生成响应模板（用于测试）"""
        return f"[{self.name}] 收到消息: {message[:50]}... \n风格: {self.style}"

# 创建模拟Agent
mock_agents = {
    "EmotionalAgent": MockAgent("EmotionalAgent", "温暖、共情、支持"),
    "TechAgent": MockAgent("TechAgent", "专业、逻辑清晰、解决问题"),
    "ToolAgent": MockAgent("ToolAgent", "高效、工具化、结果导向"),
}

# 测试响应
test_message = "我今天工作压力很大，Python代码一直报错"
selected_agent = route_message(test_message)

print(f"💬 用户消息: {test_message}")
print(f"🧭 选择Agent: {selected_agent}")
print(f"📤 响应模板:")
print(mock_agents[selected_agent].generate_response_template(test_message))

## 6. 创建自定义Agent模板

In [ ]:
# Agent模板生成器
def generate_agent_template(agent_name: str, description: str, keywords: list) -> str:
    """生成Agent模板代码"""
    return f'''
""" 
{agent_name} - {description}
"""
from typing import List, Dict, Any, Optional
from loguru import logger


class {agent_name}:
    """
    {description}
    
    触发关键词: {keywords}
    """
    
    def __init__(self, config: Dict[str, Any] = None):
        self.config = config or {{}}
        self.name = "{agent_name}"
        self.keywords = {keywords}
        logger.info(f"{{self.name}} initialized")
    
    def should_handle(self, message: str) -> bool:
        """判断是否应该处理该消息"""
        message_lower = message.lower()
        return any(kw.lower() in message_lower for kw in self.keywords)
    
    def get_system_prompt(self) -> str:
        """获取系统提示词"""
        return """
你是{agent_name}，{description}。
请根据用户的需求提供专业的帮助。
"""
    
    async def process(self, message: str, context: Dict[str, Any] = None) -> str:
        """处理消息"""
        logger.info(f"{{self.name}} processing: {{message[:50]}}...")
        # TODO: 实现具体处理逻辑
        return "处理完成"
'''

# 示例：生成研究助手Agent模板
template = generate_agent_template(
    "ResearchAgent",
    "研究助手Agent，帮助用户阅读论文、总结书籍、查找资料",
    ["论文", "研究", "文献", "总结", "书籍", "资料", "paper", "research"]
)

print("📝 生成的Agent模板:")
print("=" * 60)
print(template)

## 7. Agent性能测试

In [ ]:
import time
import random

# 生成测试消息
def generate_test_messages(count: int) -> list:
    templates = [
        "今天心情{adj}，因为{reason}",
        "帮我看看这个{lang}代码",
        "{action}一下这个数据",
        "我想了解{topic}",
    ]
    
    adj = ["不好", "很好", "一般", "糟糕"]
    reason = ["工作压力大", "考试失败", "获得了晋升", "学会了新技能"]
    lang = ["Python", "JavaScript", "Java", "Go"]
    action = ["计算", "分析", "统计", "可视化"]
    topic = ["机器学习", "区块链", "量子计算", "人工智能"]
    
    messages = []
    for _ in range(count):
        template = random.choice(templates)
        msg = template.format(
            adj=random.choice(adj),
            reason=random.choice(reason),
            lang=random.choice(lang),
            action=random.choice(action),
            topic=random.choice(topic),
        )
        messages.append(msg)
    return messages

# 性能测试
test_messages = generate_test_messages(1000)

start_time = time.time()
results = {agent: 0 for agent in AGENT_ROUTING_RULES.keys()}

for msg in test_messages:
    agent = route_message(msg)
    results[agent] += 1

elapsed = time.time() - start_time

print(f"⚡ 路由性能测试 (1000条消息):")
print(f"   总耗时: {elapsed*1000:.2f}ms")
print(f"   平均每条: {elapsed/1000*1000:.4f}ms")
print(f"\n📊 Agent分布:")
for agent, count in results.items():
    print(f"   {agent}: {count} ({count/10:.1f}%)")